# load data

In [1]:
import pandas as pd

In [115]:
test_data = "/home/s2140401/data/coco/arrows/coco_caption_karpathy_test.arrow"

In [118]:
df = pd.read_feather(test_data)
df.head()

,image,caption,image_id,split
0,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,[Friends playing Frisbee on a dry grass field ...,COCO_val2014_000000379734.jpg,test
1,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,[A bike is perched on a concrete girder on the...,COCO_val2014_000000224051.jpg,test
2,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,[A herd of cattle and several men on horses ar...,COCO_val2014_000000472732.jpg,test
3,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,[a cell phone screwdriver a pair of scissors a...,COCO_val2014_000000562843.jpg,test
4,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,[A restroom with three mirrors above two sinks...,COCO_val2014_000000545597.jpg,test


# preprocessing

In [ ]:
'''
cd /home/s2140401/script/memes_xai/models/unilm/vlmo
python make_arrow.py --root /home/s2140401/data/coco \
    --arrows_root /home/s2140401/data/coco/arrows \
'''

In [1]:
import json
import os
import pandas as pd
import pyarrow as pa
import random

from tqdm import tqdm
from glob import glob
from collections import defaultdict

In [32]:
# root = "/home/s2140401/data/coco"
root = "/home/s2140401/data/hateful_memes"

In [108]:
# with open(f"{root}/karpathy/dataset_coco.json", "r") as fp:
#     captions = json.load(fp)
# with open(f"{root}/dev.jsonl", "r") as fp:
#     captions = [json.loads(l) for l in fp.readlines()]
splits = ["train", "dev_seen", "dev_unseen", "test_seen", "test_unseen"]
split_map = {"train": "train", "dev_seen": "val", "dev_unseen": "val", "test_seen": "test", "test_unseen": "test"}
column_map = {"img": "image", "id": "image_id", "text": "caption"}
list_captions = []
for split in splits:
    jsonlpath = f"{root}/{split}.jsonl"
    caption = pd.read_json(jsonlpath, lines=True).rename(columns=column_map)
    caption["split"] = split_map[split]
    list_captions.append(caption)
df_captions = pd.concat(list_captions, ignore_index=True).dropna()
captions = df_captions.to_dict()
captions["dataset"] = "hm"


In [109]:
print(captions.keys())
print(captions["image"][0])
print(captions["caption"][0])
print(captions["split"][0])
print(captions["dataset"])

dict_keys(['image_id', 'image', 'label', 'caption', 'split', 'dataset'])
img/42953.png
its their character not their color that matters
train
hm


In [111]:
iid2captions = defaultdict(list)
iid2split = dict()
images = list(captions["image"].values())
splits = list(captions["split"].values())
texts = list(captions["caption"].values())
for image, split, text in zip(images, splits, texts):
    filename = image.split("/")[-1]
    iid2split[filename] = split
    iid2captions[filename].append(text)


In [11]:
# print(captions.keys())
# print(captions["images"][0].keys())
# print(captions["images"][0]["sentences"][0].keys())
# print(captions["dataset"])

dict_keys(['images', 'dataset'])
dict_keys(['filepath', 'sentids', 'filename', 'imgid', 'split', 'sentences', 'cocoid'])
dict_keys(['tokens', 'raw', 'imgid', 'sentid'])
coco


In [12]:
# captions = captions["images"]
# print(captions[0].keys())

dict_keys(['filepath', 'sentids', 'filename', 'imgid', 'split', 'sentences', 'cocoid'])


In [19]:
# iid2captions = defaultdict(list)
# iid2split = dict()
# for cap in tqdm(captions):
#     filename = cap["filename"]
#     iid2split[filename] = cap["split"]
#     if cap == captions[0]:
#         print(filename)
#         print(cap["split"])
#     for c in cap["sentences"]:
#         iid2captions[filename].append(c["raw"])
#         if cap == captions[0] and c == cap["sentences"][0]:
#             print(c["raw"])


 49%|████▉     | 60886/123287 [00:00<00:00, 302975.67it/s]

COCO_val2014_000000391895.jpg
test
A man with a red helmet on a small moped on a dirt road. 


100%|██████████| 123287/123287 [00:00<00:00, 305754.76it/s]


In [100]:
paths = list(glob(f"{root}/img/*.png"))
random.shuffle(paths)
print(paths[0])

/home/s2140401/data/hateful_memes/img/71439.png


In [21]:
# paths = list(glob(f"{root}/train2014/*.jpg")) + list(glob(f"{root}/val2014/*.jpg"))
# random.shuffle(paths)
# print(paths[0])

/home/s2140401/data/coco/train2014/COCO_train2014_000000445446.jpg


In [113]:
# caption_paths = [path for path in paths if path.split("/")[-1] in iid2captions]
caption_paths = [path for path in paths if path.split("/")[-1] in iid2captions]
# caption_paths = []
# cnt = 0
# for path in paths:
#     if path.split("/")[-1] in iid2captions:
#         if not cnt:
#             print(path)
#             print(path.split("/")[-1])
#             print(iid2captions[path.split("/")[-1]])
#         caption_paths.append(path)
#         cnt += 1

In [26]:
# # caption_paths = [path for path in paths if path.split("/")[-1] in iid2captions]
# caption_paths = []
# cnt = 0
# for path in paths:
#     if path.split("/")[-1] in iid2captions:
#         if not cnt:
#             print(path)
#             print(path.split("/")[-1])
#             print(iid2captions[path.split("/")[-1]])
#         caption_paths.append(path)
#         cnt += 1

/home/s2140401/data/coco/train2014/COCO_train2014_000000445446.jpg
COCO_train2014_000000445446.jpg
['A very bright colored fire hydrant by some cones.', 'a yellow and green fire hydration that is by a cone', 'There are some debris next to the fire hydrant.', 'Green and yellow fire hydrant with orange and white barrels next to it.', 'A cone laying next to a yellow and green fire hydrant. ']


In [102]:
if len(paths) == len(caption_paths):
    print("all images have caption annotations")
else:
    print("not all images have caption annotations")
print(
    len(paths), len(caption_paths), len(iid2captions),
)

all images have caption annotations
12140 12140 12140


In [27]:
# if len(paths) == len(caption_paths):
#     print("all images have caption annotations")
# else:
#     print("not all images have caption annotations")
# print(
#     len(paths), len(caption_paths), len(iid2captions),
# )

all images have caption annotations
123287 123287 123287


In [104]:
def path2rest(path, iid2captions, iid2split):
    name = path.split("/")[-1]
    with open(path, "rb") as fp:
        binary = fp.read()
    captions = iid2captions[name]
    split = iid2split[name]
    return [binary, captions, name, split]

In [105]:
bs = []
for path in tqdm(caption_paths):
    b = path2rest(path, iid2captions, iid2split)
    bs.append(b)

100%|██████████| 12140/12140 [00:58<00:00, 208.52it/s]


In [114]:
for split in ["train", "val", "restval", "test"]:
    batches = [b for b in bs if b[-1] == split]
    dataframe = pd.DataFrame(
        batches, columns=["image", "caption", "image_id", "split"],
    )
    table = pa.Table.from_pandas(dataframe)
    # os.makedirs(dataset_root, exist_ok=True)
    # with pa.OSFile(
    #     f"{dataset_root}/coco_caption_karpathy_{split}.arrow", "wb"
    # ) as sink:
    #     with pa.RecordBatchFileWriter(sink, table.schema) as writer:
    #         writer.write_table(table)

In [31]:
bs = []
for path in tqdm(caption_paths):
    name = path.split("/")[-1]
    with open(path, "rb") as fp:
        binary = fp.read()
    captions = iid2captions[name]
    split = iid2split[name]
    b = [binary, captions, name, split]
    if path == caption_paths[0]:
        print(path)
        print(binary[:10])
        print(captions[0])
        print(name)
        print(split)
    bs.append(b)

  0%|          | 589/123287 [00:00<00:39, 3109.93it/s]

/home/s2140401/data/coco/train2014/COCO_train2014_000000445446.jpg
b'\xff\xd8\xff\xe0\x00\x10JFIF'
A very bright colored fire hydrant by some cones.
COCO_train2014_000000445446.jpg
train


 67%|██████▋   | 83199/123287 [04:25<02:08, 313.10it/s] 


KeyboardInterrupt: 

In [ ]:
# for split in ["train", "val", "restval", "test"]:
#     batches = [b for b in bs if b[-1] == split]
#     dataframe = pd.DataFrame(
#         batches, columns=["image", "caption", "image_id", "split"],
#     )
#     table = pa.Table.from_pandas(dataframe)
#     os.makedirs(dataset_root, exist_ok=True)
#     with pa.OSFile(
#         f"{dataset_root}/coco_caption_karpathy_{split}.arrow", "wb"
#     ) as sink:
#         with pa.RecordBatchFileWriter(sink, table.schema) as writer:
#             writer.write_table(table)